In [3]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

## Part 1

In [20]:
class CraterDataset(Dataset):
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform

        self.image_files = os.listdir(images_dir)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = os.path.join(self.images_dir, self.image_files[idx])
        label_path = os.path.join(self.labels_dir, self.image_files[idx].replace('.jpg', '.txt'))
        
        image = Image.open(image_path).convert("RGB")
        with open(label_path, 'r') as file:
            label = torch.tensor([list(map(float, line.split())) for line in file])

        if self.transform:
            image = self.transform(image)

        return image, label

def get_dataloaders(data_dir, batch_size=16):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    train_dataset = CraterDataset(
        images_dir=os.path.join(data_dir, 'train', 'images'),
        labels_dir=os.path.join(data_dir, 'train', 'labels'),
        transform=transform
    )

    val_dataset = CraterDataset(
        images_dir=os.path.join(data_dir, 'valid', 'images'),
        labels_dir=os.path.join(data_dir, 'valid', 'labels'),
        transform=transform
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader


In [19]:
import matplotlib.pyplot as plt

def show_samples(dataloader, num_samples=5):
    for i, (images, labels) in enumerate(dataloader):
        print(i, images, labels)
        # if i >= num_samples:
        #     break
        # image = images[0].permute(1, 2, 0)  # Convert from (C, H, W) to (H, W, C)
        # plt.figure(figsize=(5, 5))
        # plt.imshow(image)
        # plt.title(f"Sample {i+1}, Labels: {labels[0].tolist()[:5]}...")  # Show first 5 label rows
        # plt.axis("off")
        # plt.show()

data_dir = "data/craters"
batch_size = 16

train_loader, val_loader = get_dataloaders(data_dir, batch_size)
show_samples(train_loader, num_samples=5)


RuntimeError: stack expects each tensor to be equal size, but got [14, 5] at entry 0 and [11, 5] at entry 1

## Part 2

In [4]:
import torch.nn as nn
import torch.optim as optim

class CraterCNN(nn.Module):
    def __init__(self):
        super(CraterCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(64 * 56 * 56, 128),
            nn.ReLU(),
            nn.Linear(128, 5)  # Output: 5 metrics per crater
        )

    def forward(self, x):
        return self.model(x)

## Part 3

In [ ]:
def train_model(model, train_loader, val_loader, epochs, lr, device):
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels.view(-1, 5))  # Flatten labels
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        val_loss = 0
        model.eval()
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels.view(-1, 5))
                val_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}")


## Part 4

In [6]:
def save_model(model, path):
    torch.save(model.state_dict(), path)

def load_model(model, path, device):
    model.load_state_dict(torch.load(path, map_location=device))
    model = model.to(device)
    return model

## Part 5

In [7]:
def main():
    data_dir = "data/craters"
    batch_size = 16
    epochs = 10
    lr = 0.001
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_loader, val_loader = get_dataloaders(data_dir, batch_size)
    model = CraterCNN()
    train_model(model, train_loader, val_loader, epochs, lr, device)

    save_model(model, os.path.join(data_dir, "best.pt"))
    print("Model training complete and saved!")

if __name__ == "__main__":
    main()

RuntimeError: stack expects each tensor to be equal size, but got [34, 5] at entry 0 and [0] at entry 1